In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ies').getOrCreate()

In [7]:
!ls /home/felipe/inep-data/data/bucket-download/censo-escolar/

2017_DOCENTES_CO.CSV	     2018_DOCENTES_CO.CSV
2017_DOCENTES_NORDESTE.CSV   2018_DOCENTES_NORDESTE.CSV
2017_DOCENTES_NORTE.CSV      2018_DOCENTES_NORTE.CSV
2017_DOCENTES_SUDESTE.CSV    2018_DOCENTES_SUDESTE.CSV
2017_DOCENTES_SUL.CSV	     2018_DOCENTES_SUL.CSV
2017_ESCOLAS.CSV	     2018_ESCOLAS.CSV
2017_MATRICULA_CO.CSV	     2018_MATRICULA_CO.CSV
2017_MATRICULA_NORDESTE.CSV  2018_MATRICULA_NORDESTE.CSV
2017_MATRICULA_NORTE.CSV     2018_MATRICULA_NORTE.CSV
2017_MATRICULA_SUDESTE.CSV   2018_MATRICULA_SUDESTE.CSV
2017_MATRICULA_SUL.CSV	     2018_MATRICULA_SUL.CSV
2017_TURMAS.CSV		     2018_TURMAS.CSV


In [2]:
!ls /home/felipe/inep-data/data/bucket-download/superior/

2017_DM_ALUNO.CSV    2017_DM_LOCAL_OFERTA.CSV	2018_DM_DOCENTE.CSV
2017_DM_CURSO.CSV    2017_TB_AUX_AREA_OCDE.CSV	2018_DM_IES.CSV
2017_DM_DOCENTE.CSV  2018_DM_ALUNO.CSV		2018_DM_LOCAL_OFERTA.CSV
2017_DM_IES.CSV      2018_DM_CURSO.CSV		2018_TB_AUX_CINE_BRASIL.CSV


In [11]:
import pandas as pd
escola_path = '../../data/bucket-download/censo-escolar/2017_MATRICULA_SUDESTE.CSV'
ies_path = '../../data/bucket-download/superior/2018_DM_ALUNO.CSV'
display(pd.read_csv(escola_path, nrows=1, sep = '|').columns[0:10])
display(pd.read_csv(ies_path, nrows=1, sep = '|').columns[0:10])

Index(['NU_ANO_CENSO', 'ID_MATRICULA', 'CO_PESSOA_FISICA', 'NU_DIA', 'NU_MES',
       'NU_ANO', 'NU_IDADE_REFERENCIA', 'NU_IDADE', 'NU_DURACAO_TURMA',
       'NU_DUR_ATIV_COMP_MESMA_REDE'],
      dtype='object')

Index(['NU_ANO_CENSO', 'CO_IES', 'TP_CATEGORIA_ADMINISTRATIVA',
       'TP_ORGANIZACAO_ACADEMICA', 'CO_CURSO', 'CO_CURSO_POLO', 'TP_TURNO',
       'TP_GRAU_ACADEMICO', 'TP_MODALIDADE_ENSINO', 'TP_NIVEL_ACADEMICO'],
      dtype='object')

In [ ]:
df = {}

df['2017_MATRICULA_SUDESTE'] = spark.read.csv(escola_path,
                    inferSchema = True,
                    header = True,
                    encoding='ISO-8859-1',
                    sep = '|').select(['CO_PESSOA_FISICA'])

df['2018_DM_ALUNO'] = spark.read.csv(ies_apth,
                    inferSchema = True,
                    header = True,
                    encoding='ISO-8859-1',
                    sep = '|').select(['ID_ALUNO','CO_IES'])

In [ ]:
from pyspark.sql.functions import isnan, when, count, col
nulls = df['2018_DM_ALUNO'].select([count(when(isnan(c) | col(c).isNull(), c)
                                         ).alias(c) for c in df['2018_DM_ALUNO'].columns]
                                  ).collect()
nulls